## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atar,MR,20.52,-13.05,89.94,12,99,11.39,overcast clouds
1,1,Mar Del Plata,AR,-38.00,-57.56,69.80,49,20,9.17,few clouds
2,2,Lebu,CL,-37.62,-73.65,58.77,76,8,7.47,clear sky
3,3,Ushuaia,AR,-54.80,-68.30,59.00,54,75,4.70,light rain
4,4,Nuuk,GL,64.18,-51.72,24.80,73,100,2.24,light snow


In [7]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [8]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Atar,MR,20.52,-13.05,89.94,12,99,11.39,overcast clouds
6,6,Filingue,NE,14.35,3.32,89.47,12,18,7.38,few clouds
8,8,Akyab,MM,20.15,92.90,77.86,87,98,7.94,overcast clouds
9,9,Tautira,PF,-17.73,-149.15,80.60,74,75,2.24,shower rain
11,11,Boa Vista,BR,2.82,-60.67,82.40,69,75,16.11,shower rain
12,12,Comodoro Rivadavia,AR,-45.87,-67.50,80.60,36,0,5.82,clear sky
14,14,Saldanha,ZA,-33.01,17.94,78.80,24,0,6.93,clear sky
20,20,Puerto Ayora,EC,-0.74,-90.35,77.00,69,75,19.46,broken clouds
28,28,Samarai,PG,-10.62,150.67,81.09,83,24,7.36,few clouds
29,29,Atambua,ID,-9.11,124.89,75.29,79,69,2.84,broken clouds


In [9]:
# 4a. Determine if there are any empty rows.

preferred_cities_df.count()

Unnamed: 0             199
City                   199
Country                197
Lat                    199
Lng                    199
Max Temp               199
Humidity               199
Cloudiness             199
Wind Speed             199
Current Description    199
dtype: int64

In [12]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_preferred_cities_df = preferred_cities_df.dropna()


In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Atar,MR,89.94,overcast clouds,20.52,-13.05,
6,Filingue,NE,89.47,few clouds,14.35,3.32,
8,Akyab,MM,77.86,overcast clouds,20.15,92.90,
9,Tautira,PF,80.60,shower rain,-17.73,-149.15,
11,Boa Vista,BR,82.40,shower rain,2.82,-60.67,
12,Comodoro Rivadavia,AR,80.60,clear sky,-45.87,-67.50,
14,Saldanha,ZA,78.80,clear sky,-33.01,17.94,
20,Puerto Ayora,EC,77.00,broken clouds,-0.74,-90.35,
28,Samarai,PG,81.09,few clouds,-10.62,150.67,
29,Atambua,ID,75.29,broken clouds,-9.11,124.89,


In [20]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
        
    # 6c. Get latitude and longitude from DataFrame
        lat = row["Lat"]
        lng = row["Lng"]
        
        params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
        hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    
        try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        except (IndexError):
            print("Hotel not found... skipping.")   
  

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [22]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [24]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [31]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure

fig


Figure(layout=FigureLayout(height='420px'))